In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
    
    def forward(self, x):
        x = self.conv(x)
        return F.relu(x, inplace=True)

In [4]:
class InceptionV1(nn.Module):
    def __init__(self, in_dim, hid_1x1, hid_2x1, hid_2x3, hid_3x1, out_3x5, out_4x1):
        super(InceptionV1, self).__init__()
        self.branch_1x1 = BasicConv2d(in_dim, hid_1x1, 1)
        self.branch_3x3 = nn.Sequential(
            BasicConv2d(in_dim, hid_2x1, 1),
            BasicConv2d(hid_2x1, hid_2x3, 3, padding=1)
        )
        self.branch_5x5 = nn.Sequential(
            BasicConv2d(in_dim, hid_3x1, 1),
            BasicConv2d(hid_3x1, out_3x5, 5, padding=2)
        )
        self.branch_pooling = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            BasicConv2d(in_dim, out_4x1, 1)
        )
        
    def forward(self, x):
        b1 = self.branch_1x1(x)
        b2 = self.branch_3x3(x)
        b3 = self.branch_5x5(x)
        b4 = self.branch_pooling(x)
        out = torch.cat((b1, b2, b3, b4), dim=1)
        return out

In [5]:
inception = InceptionV1(3, 64, 32, 64, 64, 96, 32)

In [6]:
inception

InceptionV1(
  (branch_1x1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (branch_3x3): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (branch_5x5): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicConv2d(
      (conv): Conv2d(64, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
  )
  (branch_pooling): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (1): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

In [7]:
data = torch.randn(1, 3, 256, 256)

In [8]:
out = inception(data)
out.shape

torch.Size([1, 256, 256, 256])